In [41]:
import numpy as np
import pandas as pd


import matplotlib.pyplot as plt

import seaborn as sns
import plotly.express as px
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool, Range1d, Circle, Line, TapTool, CustomJS
from bokeh.plotting import figure

In [15]:
df = pd.read_csv("Sample - Superstore.csv", index_col = 'Row ID', encoding='cp1252')
df.head(3)

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
Row ID,,,,,,,,,,,,,,,,,,,,
1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820
3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.62,2,0.0,6.8714


## Biểu đồ 1

In [48]:

chart_df = df.copy()
top_10_cities = chart_df['City'].value_counts().nlargest(10)
top_10_cities.sort_values(ascending=True, inplace=True)


# Enable inline visualization in Jupyter Notebook
output_notebook(hide_banner=True)

# Define the data for the lollipop chart
x_values = list(top_10_cities.values)
y_values = np.arange(10)
# Calculate the x and y coordinates for the line
line_x = [0] * len(x_values)
line_y = [y_values[0]] * len(y_values)
colors = ['#F7D060','#ADE4DB','#957777','#FF6969','#394867','#A5C0DD','#F79540', '#B71375', '#B71375', '#8B1874']

# Add the x and y coordinates for the line to the data dictionary
data = {'x': x_values, 'y': y_values, 'line_x': line_x, 'line_y': line_y, 'colors': colors}

# Create a ColumnDataSource object to hold the data
source = ColumnDataSource(data)

# Create a new figure
y_range = Range1d(start=-1, end=10)
p = figure(height=600, width=800, title='Horizontal Lollipop Chart', y_range=y_range, tools='tap')
p.yaxis.ticker = y_values
p.yaxis.major_label_overrides = dict(zip(y_values, top_10_cities.index))


# Draw the horizontal lollipop chart
glyph = Circle(x='x', y='y', size=10, fill_color='colors', line_color='white')
renderer = p.add_glyph(source, glyph)
p.segment(x0=0, y0='y', x1='x', y1='y',line_color='colors', source=source)

line_glyph = Line(x='line_x', y='line_y', line_color='black', line_width=3, line_alpha=1)
p.add_glyph(source, line_glyph)

# Add tooltips to display the data
hover = HoverTool(tooltips=[('X', '@x'), ('Y', '@y')], renderers=[renderer])
p.add_tools(hover)

# Define the callback function to handle click events
callback = CustomJS(args=dict(line_glyph=line_glyph), code="""
    var selected_index = cb_obj.indices[0];
    var selected_glyph = cb_obj.selection_glyph;
    selected_glyph.size = 20;
    selected_glyph.line_color = 'red';
    line_glyph.line_alpha = 1;
    line_glyph.data_source.data.line_x = [0, cb_obj.data.x[selected_index]];
    line_glyph.data_source.data.line_y = [cb_obj.data.y[selected_index], cb_obj.data.y[selected_index]];
    line_glyph.data_source.change.emit();
""")

# Add the callback function to the TapTool
taptool = p.select(type=TapTool)
taptool.callback = callback

# Display the chart
show(p)